# 多模态端侧小模型-GLM-Edge-v-5b 部署流程

>本内容包括GLM-Edge系列模型的快速介绍以及本地部署的流程。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241203115131325.png" width=70%></div>

## 模型简介

GLM-Edge 是清华大学（智谱轻言）推出的一系列专为端侧设备设计的大语言模型和多模态理解模型。这些模型包括 GLM-Edge-1.5B-Chat、GLM-Edge-4B-Chat、GLM-Edge-V-2B 和 GLM-Edge-V-5B。其中，1.5B和2B模型主要针对手机、车机等轻量化设备平台，而4B和5B模型则更适用于PC等高性能平台。


GLM-Edge系列模型在追求高效性能的同时，特别注重实际部署的便捷性与推理速度优化，在高通骁龙和Intel平台上的表现尤为出色，展现了卓越的适配能力与运行效率。并支持多种推理后端（如 transformers、OpenVINO、vLLM）进行模型推理，同时官方提供微调教程和配置文件，支持用户根据具体需求进行模型微调。

例如，在高通骁龙8 Elite平台上，GLM-Edge凭借其强大的NPU算力，通过混合量化方案使1.5B对话模型和2B多模态模型的解码速度达到每秒60 tokens以上。在引入投机采样技术后，这两款模型的解码速度峰值可超过每秒100 tokens。这些高效推理方案将在后续由GLM团队或其合作伙伴陆续发布。

**部署流程细节可以扫描下放二维码领取完整课件同步进行学习**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/bc6d74d18490f68600356f4d68927d7.jpg" width=70%></div>

# 本地部署流程

> 本流程是在Linux环境下通过命令行进行部署的，推理图象识别多模态模型GLM-Edge-v-5b需要至少19G显存。

### 本地部署硬件推荐

Linux系统：
1. 操作系统：推荐Ubuntu 22.04或更高版本，或其他支持的Linux发行版。
2. 显卡:推荐使用 NVIDIA GPU（如 T4、V100、A10 等），或 AMD GPU，在本流程中至少需要 22GB 的显存。
3. 内存：至少 16GB 的 RAM，在运行更大的模型的时候需要更多内存。4. 硬盘空间：在每个模型的支持列表中会显示安装所需的空间大小，在本流程中最少10GB用于模型下载。

- **Step 1. 创建conda虚拟环境**

&emsp;&emsp;Conda创建虚拟环境的意义在于提供了一个隔离的、独立的环境，用于Python项目和其依赖包的管理。每个虚拟环境都有自己的Python运行时和一组库。这意味着我们可以在不同的环境中安装不同版本的库而互不影响。根据官方文档信息建议Python版本3.10以上。创建虚拟环境的办法可以通过使用以下命令创建：

```bash
# GLM4 是你想要给环境的名称，python=3.11 指定了要安装的Python版本。你可以根据需要选择不同的名称和/或Python版本。

conda create -n glm4 python=3.11
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240924162412814.png" width=80%></div>

&emsp;&emsp;创建虚拟环境后，需要激活它。使用以下命令来激活刚刚创建的环境。如果成功激活，可以看到在命令行的最前方的括号中，就标识了当前的虚拟环境。虚拟环境创建完成后接下来安装torch。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240924162442522.png" width=80%></div>

&emsp;&emsp;如果忘记或者想要管理自己建立的虚拟环境，可以通过conda env list命令来查看所有已创建的环境名称。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240928143512354.png" width=80%></div>

&emsp;&emsp;如果需要卸载指定的虚拟环境则通过以下指令实现：
```
conda env remove --name envname
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240928143643575.png" width=80%></div>

- 需要注意的是无法卸载当前激活的环境，建议卸载时先切换到base环境中再执行操作。

- **Step 2. 查看当前驱动最高支持的CUDA版本**

&emsp;&emsp;我们需要根据CUDA版本选择Pytorch框架，先看下当前的CUDA版本：
```
nvidia -smi
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240924161818464.png" width=70%></div>

- **Step 3. 在虚拟环境中安装Pytorch**

&emsp;&emsp;进入Pytorch官网：https://pytorch.org/get-started/previous-versions/

<div align=center><img src="https://muyu001.oss-cn-beijing.aliyuncs.com/img/image-20240103163206436.png" width=80%></div>

&emsp;&emsp;当前的电脑CUDA的最高版本要求是12.2，所以需要找到不大于12.2版本的Pytorch。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401041455184.png" width=80%></div>

&emsp;&emsp;直接复制对应的命令，进入终端执行即可。这实际上安装的是为 CUDA 12.1 优化的 PyTorch 版本。这个 PyTorch 版本预编译并打包了与 CUDA 12.1 版本相对应的二进制文件和库。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240924163535040.png" width=80%></div>

- **Step 4. 安装Pytorch验证**

&emsp;&emsp;待安装完成后，如果想要检查是否成功安装了GPU版本的PyTorch，可以通过几个简单的步骤在Python环境中进行验证：
```bash
import torch

print(torch.__version__)
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240924164006735.png" width=80%></div>

&emsp;&emsp;如果输出是版本号数字，则表示GPU版本的PyTorch已经安装成功并且可以使用CUDA，如果显示ModelNotFoundError，则表明没有安装GPU版本的PyTorch，或者CUDA环境没有正确配置，此时根据教程，重新检查自己的执行过程。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240924164131600.png" width=80%></div>

&emsp;&emsp;当然通过pip show的方式可以很简洁的查看已安装包的详细信息。pip show <package_name> 可以展示出包的版本、依赖关系（展示一个包依赖哪些其他包）、定位包安装位置、验证安装确实包是否正确安装及详情。

- **Step 5. 创建文件夹**

可以通过下图的方式创建新的文件夹`mkdir glm-edge`,其中，glm-edge为自定义的文件夹名称。

创建好文件后通过`cd glm-edge`的命令进入到指定的文件地址中。

通过指令`pwd`可以展示当前文件夹的绝对路径，稍后我们需要该信息指定我们的下载地址。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241203111553941.png" width=80%></div>

- **Step 6. 从Model Scope下载GLM-Edge-v-5b模型**

&emsp;&emsp;本次介绍的是 Model Scope 下载方法。Model Scope 是一个国内平台的资源丰富的模型库，开发者可以上传和共享他们训练好的机器学习模型。这些模型通常是经过大量数据训练的，并且很大，因此需要特殊的存储和托管服务。


&emsp;&emsp;Model Scope下载路径如下：
https://www.modelscope.cn/models/ZhipuAI/glm-edge-v-5b/files


&emsp;&emsp;Model Scope支持多种下载模式：SDK下载、Git下载、命令行下载（下载完整模型、下载单个文件 后直接加文件名即可）、手动下载

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241203111000355.png" width=60%></div>

- **Step 7. 命令行方式下载**

这是一种快捷的本机下载方式，首先通过命令`pip install modelscope`安装好达摩社区的安装包，然后通过以下指令可以下载模型的权重文件，这种方法无需翻墙，无需下载额外的插件。
```
pip install modelscope
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240924184742785.png" width=70%></div>

通过以下命令实现指定下载路径安装完整的文件，需要注意的是`./dir`需要替换成你本地想要储存的文件路径。
```
modelscope download --model ZhipuAI/glm-edge-v-5b  --local_dir ./dir
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241202135920636.png" width=70%></div>

下载完成后全部的权重文件的内容如下：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241202135931314.png" width=70%></div>

- **Step 8. 模型使用测试**

我们可以在安装模型的对应文件夹内创建一个新的脚本进行模型推理测试，通过`vim run.py`来创建一个名为run的python文件。

随后在新建的文件中复制以下代码，需要注意的是，要对代码中图像输入的部分修改成你在本地保存的图像文件的绝对路径，并将model_dir后的信息修改成模型存储的绝对路径。

```python
import torch
from PIL import Image
from modelscope import snapshot_download
from transformers import (
    AutoTokenizer,
    AutoImageProcessor,
    AutoModelForCausalLM,
)

url = "/home/data/LLM/glm-edge/tb.png" # 这里需要修改成你在本地保存的图像文件的绝对路径
messages = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": "describe this image"}]}]
image = Image.open(url)

model_dir = "/home/data/LLM/glm-edge" # 这里需要修改成模型所在文件夹的绝对路径

processor = AutoImageProcessor.from_pretrained(model_dir, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

inputs = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_dict=True, tokenize=True, return_tensors="pt"
).to(next(model.parameters()).device)

generate_kwargs = {
    **inputs,
    "pixel_values": torch.tensor(processor(image).pixel_values).to(next(model.parameters()).device),
}
output = model.generate(**generate_kwargs, max_new_tokens=100)
print(tokenizer.decode(output[0][len(inputs["input_ids"][0]):], skip_special_tokens=True))


```

完成好修改后，保存并退出。通过`python run.py`命令启动该文件，实现模型的多模态图像识别推理。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241202165059946.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20241203114909038.png" width=100%></div>